In [1]:
from fastai.vision.all import *
import torchaudio
import librosa
from nnAudio.features.cqt import CQT1992v2
from scipy.signal import tukey


C:\Users\anaconda\envs\Pytorch_19\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CFG:
    apex=False
    debug=False
    print_freq=100
    num_workers=4
    model_name='tf_efficientnet_b7_ns'
    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=3
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=3 # CosineAnnealingLR
    #T_0=3 # CosineAnnealingWarmRestarts
    lr=1e-4
    min_lr=1e-6
    batch_size=48
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    qtransform_params={"hop_length": 1024, "n_bins": 64,'fmax':None}    
    seed=42
    target_size=1
    target_col='target'
    n_fold=5
    trn_fold=[0] # [0, 1, 2, 3, 4]
    train=True
    grad_cam=False


In [3]:
df_train = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/train.csv')
df_test = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/test.csv')
submission = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/sample_submission.csv')

In [4]:
train_path = Path('C:/Users/Kaggle/Pog_Music_Classification/train/')
test_path = Path('C:/Users/Kaggle/Pog_Music_Classification/test/')

In [5]:
print(df_train.shape,df_test.shape)

(19922, 5) (5078, 3)


In [6]:
df_train.head()

,song_id,filename,filepath,genre_id,genre
0,10150,010150.ogg,train/010150.ogg,7,Instrumental
1,7358,007358.ogg,train/007358.ogg,2,Punk
2,20573,020573.ogg,train/020573.ogg,5,Folk
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic
4,16662,016662.ogg,train/016662.ogg,1,Rock


In [7]:
import glob
valid_filenames = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/train/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames.append(fnames.split('.')[0]+'.ogg')

In [8]:
import glob
valid_filenames_test = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/test/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames_test.append(fnames.split('.')[0]+'.ogg')

In [9]:
valid_filenames

['000000.ogg',
 '000001.ogg',
 '000002.ogg',
 '000004.ogg',
 '000005.ogg',
 '000007.ogg',
 '000009.ogg',
 '000010.ogg',
 '000012.ogg',
 '000013.ogg',
 '000014.ogg',
 '000015.ogg',
 '000016.ogg',
 '000018.ogg',
 '000019.ogg',
 '000020.ogg',
 '000021.ogg',
 '000022.ogg',
 '000025.ogg',
 '000026.ogg',
 '000027.ogg',
 '000028.ogg',
 '000029.ogg',
 '000030.ogg',
 '000033.ogg',
 '000034.ogg',
 '000035.ogg',
 '000037.ogg',
 '000039.ogg',
 '000040.ogg',
 '000041.ogg',
 '000042.ogg',
 '000043.ogg',
 '000045.ogg',
 '000046.ogg',
 '000047.ogg',
 '000049.ogg',
 '000051.ogg',
 '000052.ogg',
 '000053.ogg',
 '000054.ogg',
 '000055.ogg',
 '000056.ogg',
 '000058.ogg',
 '000059.ogg',
 '000061.ogg',
 '000062.ogg',
 '000063.ogg',
 '000066.ogg',
 '000068.ogg',
 '000069.ogg',
 '000070.ogg',
 '000071.ogg',
 '000072.ogg',
 '000073.ogg',
 '000074.ogg',
 '000075.ogg',
 '000076.ogg',
 '000077.ogg',
 '000078.ogg',
 '000080.ogg',
 '000081.ogg',
 '000082.ogg',
 '000083.ogg',
 '000084.ogg',
 '000085.ogg',
 '000087.o

In [10]:
df_train = df_train[df_train['filename'].isin(valid_filenames)].reset_index(drop=True)
df_test_valid = df_test[df_test['filename'].isin(valid_filenames_test)].reset_index(drop=True)

In [11]:
### Some files giving exception in cqt
df_train = df_train[df_train['song_id']!=17400].reset_index(drop=True)

In [12]:
print(df_train.shape,df_test.shape)

(19908, 5) (5078, 3)


### Upsample minority classes

In [13]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

In [14]:
ExtremeMinority = [18]
SomeMinority = [17]
LowMinority = [14,15,16]

# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(ExtremeMinority)].sample(n=37,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(SomeMinority)].sample(n=42,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(LowMinority)].sample(n=50,replace=True,random_state=42)],0).reset_index(drop=True)

In [15]:
df_train.tail()#,df_train.shape

,song_id,filename,filepath,genre_id,genre
19903,9337,009337.ogg,train/009337.ogg,0,Electronic
19904,8340,008340.ogg,train/008340.ogg,4,Hip-Hop
19905,16248,016248.ogg,train/016248.ogg,4,Hip-Hop
19906,11875,011875.ogg,train/011875.ogg,2,Punk
19907,6579,006579.ogg,train/006579.ogg,1,Rock


In [16]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

### Create Folds

In [17]:
df_train['fold'] = 0

In [18]:
from sklearn.model_selection import StratifiedKFold
N_folds = 5
kf = StratifiedKFold(5,random_state=42,shuffle=True)
fld = 0
for i,(_,val_id) in enumerate(kf.split(df_train['filename'],df_train['genre_id'])):
    df_train.loc[val_id,'fold'] = int(fld)
    fld += 1

In [19]:
df_train.head()

,song_id,filename,filepath,genre_id,genre,fold
0,10150,010150.ogg,train/010150.ogg,7,Instrumental,4
1,7358,007358.ogg,train/007358.ogg,2,Punk,3
2,20573,020573.ogg,train/020573.ogg,5,Folk,2
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic,3
4,16662,016662.ogg,train/016662.ogg,1,Rock,3


In [20]:
df_test.head()

,song_id,filename,filepath
0,7072,007072.ogg,test/007072.ogg
1,10207,010207.ogg,test/010207.ogg
2,20008,020008.ogg,test/020008.ogg
3,10924,010924.ogg,test/010924.ogg
4,21896,021896.ogg,test/021896.ogg


#### Channel wise max: 2.7517, 2.7950

#### Dataset

In [21]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.transforms import Compose
import torchvision

tfms = Compose(
#     [torchvision.transforms.Resize((128,128))])
    [torchvision.transforms.CenterCrop((64,448))])


In [22]:
import torch_audiomentations as tA

def get_transforms(*, data):
    
    if data == 'train':
        return tA.Compose(
                transforms=[
                     tA.ShuffleChannels(p=0.1,mode="per_example",p_mode="per_example", sample_rate= 16000),
                    tA.PeakNormalization(p=0.5,apply_to='only_too_loud_sounds'),
#                     tA.BandPassFilter()
#                      tA.AddColoredNoise(p=0.1,mode="per_channel",p_mode="per_channel", sample_rate=16000,max_snr_in_db = 15),
                     tA.Shift(p=0.1,mode="per_example",p_mode="per_example", sample_rate=16000,
                              max_shift=0.025, min_shift=-0.025),
                ])

    elif data == 'valid':
        return tA.Compose([
        ])

In [23]:
class POGFiles(torch.utils.data.Dataset):
    
    def __init__(self, df, 
                 train = True,
                 augmentations = None,
                 normalize=False,
                 ch0Max=2.7517,
                 ch1Max=2.7950,
                 HOP_LEN = 1024,
                 N_FFT=1024,
                 n_mels=224,
                 transform=tfms,
                 audiotfms=None,
                 applyTukey=False,
                 quantizeData=False,
                 MakeThreeChannel= False,
                 bins_per_octave = 24,
                 n_bins = 128
                ):
        
        
        self.df = df
        self.train = train
        self.ch0Max = ch0Max
        self.ch1Max = ch1Max        
        self.N_FFT = N_FFT
        self.n_bins = n_bins
        self.bins_per_octave = bins_per_octave
        self.HOP_LEN = HOP_LEN
        self.transform = transform
        self.n_mels= n_mels
        self.audiotfms = audiotfms
        self.normalize = normalize
        self.MakeThreeChannel = MakeThreeChannel
        
    def __len__(self):
        return len(self.df)
    
    def load_file(self, filename):
        if self.train:
            audio = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/train/{filename}')
        else:
            audio = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/test/{filename}')
            
        audio = np.mean(audio, axis=0, keepdims=True)   ### convert to mono
        return audio

    def resample(self,audio,sr):
        
        audio = torchaudio.transforms.Resample(orig_freq= sr, new_freq=  16000, 
                                    resampling_method = 'sinc_interpolation', 
                                    lowpass_filter_width = 6, 
                                    rolloff = 0.99)(audio)
        return audio

    def makeCQT(self,audio,sr):
        audio = CQT1992v2(hop_length=self.HOP_LEN,n_bins=self.n_bins,bins_per_octave=self.bins_per_octave,
                          sr=sr,verbose=False)(audio)
        return audio
        
        
    def create_spectrogram(self,audio,sr):
                
        specgram = torchaudio.transforms.MelSpectrogram(sample_rate=sr,
                                                        n_fft=self.N_FFT, 
                                                        win_length=self.N_FFT, 
                                                        hop_length=self.HOP_LEN,
                                                        center=True,
                                                        pad_mode="reflect",
                                                        power=2.0,
                                                        norm='slaney',
                                                        onesided=True,
                                                        n_mels=self.n_mels,
                                                        mel_scale="htk"
                                                       )(audio)

        specgram = torchaudio.transforms.AmplitudeToDB()(specgram)
        specgram = specgram - specgram.min()
        
        max_ = specgram.max()
        if max_ < 0.001:
            specgram[...] = 0
        else:
            specgram /= max_
        return specgram

    def __getitem__(self, index):
        row  = self.df.iloc[index]
        
        sr = 16000
        samples = self.load_file(row.filename.split('.')[0]+'.npy')
#         samples = self.load_file(row.filename)
    
        samples = np.stack(samples, axis=0)
        
        
        samples = torch.from_numpy(samples).float().view(1,-1)
        
        if self.audiotfms is not None:
             samples = self.audiotfms(samples.unsqueeze(0)).squeeze(0)

        samples = self.create_spectrogram(samples,sr)
                
        if self.transform is not None:
            samples = self.transform(samples)

            
        if self.train:
            label = torch.tensor(row.genre_id,dtype=torch.long)
            return samples, label
        else:

            return samples

In [24]:
# class POGFiles(torch.utils.data.Dataset):
    
#     def __init__(self, df, 
#                  train = True,
#                  augmentations = None,
#                  normalize=False,
#                  ch0Max=2.7517,
#                  ch1Max=2.7950,
#                  HOP_LEN = 1024,
#                  N_FFT=1024,
#                  n_mels=224,
#                  transform=tfms,
#                  audiotfms=None,
#                  applyTukey=False,
#                  quantizeData=False,
#                  MakeThreeChannel= False,
#                  bins_per_octave = 24,
#                  n_bins = 128
#                 ):
        
        
#         self.df = df
#         self.train = train
#         self.ch0Max = ch0Max
#         self.ch1Max = ch1Max        
#         self.N_FFT = N_FFT
#         self.n_bins = n_bins
#         self.bins_per_octave = bins_per_octave
#         self.HOP_LEN = HOP_LEN
#         self.transform = transform
#         self.n_mels= n_mels
#         self.audiotfms = audiotfms
#         self.normalize = normalize
#         self.applyTukey = applyTukey
#         self.quantizeData = quantizeData
#         self.MakeThreeChannel = MakeThreeChannel
        
#     def __len__(self):
#         return len(self.df)
    
#     def load_file(self, filename):
#         if self.train:
#             audio = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/train/{filename}')
#         else:
#             audio = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/test/{filename}')
#         return audio
    
#     def apply_preprocess(self, waves):
#         samples = np.hstack(waves)
#         samples = waves / np.max(waves)
#         return waves
        
#     def quantize_data(self, data, classes):
#         mu_x = self.mu_law_encoding(data, classes)
#         return mu_x

#     def mu_law_encoding(self,data, mu):
#         mu_x = np.sign(data) * np.log(1 + mu * np.abs(data)) / np.log(mu + 1)
#         return mu_x

#     def resample(self,audio,sr):
        
#         audio = torchaudio.transforms.Resample(orig_freq= sr, new_freq=  16000, 
#                                     resampling_method = 'sinc_interpolation', 
#                                     lowpass_filter_width = 6, 
#                                     rolloff = 0.99)(audio)
#         return audio

#     def makeCQT(self,audio,sr):
#         audio = CQT1992v2(hop_length=self.HOP_LEN,n_bins=self.n_bins,bins_per_octave=self.bins_per_octave,
#                           sr=sr,verbose=False)(audio)
#         return audio
        
        
#     def create_spectrogram(self,audio):
                
#         specgram = torchaudio.transforms.MelSpectrogram(sample_rate=16000,
#                                                         n_fft=self.N_FFT, 
#                                                         win_length=self.N_FFT, 
#                                                         hop_length=self.HOP_LEN,
#                                                         center=True,
#                                                         pad_mode="reflect",
#                                                         power=2.0,
#                                                         norm='slaney',
#                                                         onesided=True,
#                                                         n_mels=self.n_mels,
#                                                         mel_scale="htk"
#                                                        )(audio)

#         specgram = torchaudio.transforms.AmplitudeToDB()(specgram)
# #         specgram = specgram - specgram.min()
# #         specgram = specgram/specgram.max()*255
#         return specgram

#     def __getitem__(self, index):
#         row  = self.df.iloc[index]

#         samples = self.load_file(row.filename.split('.')[0]+'.npy')
        
#         if samples.shape[0] != 2:
#             samples = np.concatenate((samples,samples),0)
#             assert samples.shape[0] == 2      
    
#         samples = np.stack(samples, axis=0)
        
#         if self.applyTukey:
#             samples = samples*scipy.signal.tukey(samples.shape[-1],0.2)
        
#         if self.MakeThreeChannel:
#             mean = samples.mean(0).reshape(1,-1)
# #             diff = samples[0,:,:]-samples[1,:,:]
#             samples = np.vstack((samples,mean))    
#             samples = torch.from_numpy(samples).float().view(3,-1)
#         else:
#             samples = torch.from_numpy(samples).float().view(2,-1)
        
#         if self.normalize:
#             samples = samples/torch.abs(samples).max()
                
#         if self.audiotfms is not None:
#              samples = self.audiotfms(samples.unsqueeze(0)).squeeze(0)

#         samples = self.makeCQT(samples,sr=16000)
        
# #         samples = self.create_spectrogram(samples)
    
#         if self.transform is not None:
#             samples = self.transform(samples)

#         if self.quantizeData:
#             samples = self.quantize_data(samples,19)
            
#         if self.train:
#             label = torch.tensor(row.genre_id,dtype=torch.long)
#             return samples, label
#         else:

#             return samples

In [25]:
# a,b = next(iter(POGFiles(df_train)))
# a.shape

In [26]:
# # a,b = next(iter(POGFiles(df_train,transform=None,n_mels=128,HOP_LEN=512,N_FFT=2048)))
a,b = next(iter(POGFiles(df_train,transform=tfms,n_mels=64,HOP_LEN=1024,N_FFT=512)))
a.shape

torch.Size([1, 64, 448])

In [27]:
def gem_1d_res(x, p=3, eps=1e-6):
    return F.avg_pool1d(x.clamp(min=eps).pow(p), 2,2).pow(1./p)

def gem_1d(x, p=3, eps=1e-6):
    return F.avg_pool1d(x.clamp(min=eps).pow(p), (x.size(-1),)).pow(1./p )

#### FastAI model

In [28]:
df_train['genre_id'].nunique()

19

In [29]:
df_train.head()

,song_id,filename,filepath,genre_id,genre,fold
0,10150,010150.ogg,train/010150.ogg,7,Instrumental,4
1,7358,007358.ogg,train/007358.ogg,2,Punk,3
2,20573,020573.ogg,train/020573.ogg,5,Folk,2
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic,3
4,16662,016662.ogg,train/016662.ogg,1,Rock,3


In [30]:
import timm
timm.list_models()

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'botnet26t_256',
 'botnet50ts_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_base',
 'convnext_base_384_in22ft1k',
 'convnext_base_in22ft1k',
 'convnext_base_in22k',
 'convnext_large',
 'convnext_large_384_in22ft1k',
 'convnext_large_in22ft1k',
 'convnext_large_in22k',
 'convnext_small',
 'convnext_tiny',
 'convnext_tiny_hnf',
 'convnext_xlarge_384_in22ft1k',
 'convnext_xlarge_in22ft1k',
 'convnext_x

In [31]:
# timm.create_model('tf_efficientnet_b1')

In [32]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

In [33]:
df_train.head()

,song_id,filename,filepath,genre_id,genre,fold
0,10150,010150.ogg,train/010150.ogg,7,Instrumental,4
1,7358,007358.ogg,train/007358.ogg,2,Punk,3
2,20573,020573.ogg,train/020573.ogg,5,Folk,2
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic,3
4,16662,016662.ogg,train/016662.ogg,1,Rock,3


In [34]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM,self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'

In [35]:
def run():
    oof = df_train.copy()
    oversampleTrain = False
    
    for fold_num in [0,1,2,3,4]:
        print('*****************************************')
        print(f'Training Fold {fold_num}')
        print('*****************************************')

    #with IPyExperimentsPytorch() as exp:
        kernel_type = 'tf_efficientnet_b1_MonoSpec'
        OUTPUT_DIR = f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/'
        if not os.path.exists(OUTPUT_DIR):
            os.makedirs(OUTPUT_DIR)

            
        crit = CrossEntropyLossFlat() #LabelSmoothingCrossEntropy(0.025) #CrossEntropyLossFlat() #FocalLoss()
        batch_size = 64
        n_epochs = 7

        training_fold = df_train.query(f'fold!={fold_num}').reset_index(drop=True, inplace=False)
        
        if oversampleTrain:
            print('--------- Oversampling training dataset -----------')
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(ExtremeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(SomeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(LowMinority)].sample(n=100,replace=True,random_state=42)],0).reset_index(drop=True)
                
        train_ds = POGFiles(training_fold,
                            transform=tfms,n_mels=64,HOP_LEN=1024,N_FFT=512,
                            )

        validation_fold = df_train.query(f'fold=={fold_num}').reset_index(drop=True, inplace=False)
        valid_ds = POGFiles(validation_fold,
                            transform=tfms,n_mels=64,HOP_LEN=1024,N_FFT=512,
                            )

        print(f'- Training samples: {len(train_ds)}\n- Validation Samples : {len(valid_ds)}')

        bs = batch_size
        train_dl = torch.utils.data.DataLoader(train_ds, batch_size=bs, num_workers=0,pin_memory=False)
        valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=bs*2, num_workers=0,shuffle=False,pin_memory=False)

        dls = DataLoaders(train_dl, valid_dl)
        model = timm.create_model('tf_efficientnet_b1', pretrained=False, in_chans=1)#get_model(resnet34)
#         model.conv1.stride = (1,1)
#         model.global_pool = GeM()
        model.conv_stem.stride = (1,1)
        model.classifier = nn.Linear(1280,19)
        f1_score = F1Score(average="micro")
        
        ### MixUp() in callbacks
#        ReduceLROnPlateau(monitor='valid_loss', min_delta=0.1,patience=2,min_lr=1e-7)

        learn = Learner(dls, model,opt_func=ranger,loss_func=crit,metrics=[f1_score],cbs=[SaveModelCallback('f1_score', every_epoch=True),
                                                           CSVLogger(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/{fold_num}logs.csv')])#.to_fp16()
          
#         learn.fit(n_epochs, 1e-2)       
#         learn.fit_one_cycle(n_epochs, 1e-3, wd=1e-03)
        learn.fit_flat_cos(n_epochs, 1e-2, wd=1e-03)

        learn = learn.to_fp16()
        learn.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')

        learn = learn.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        learn.model.eval()
        test_df = df_test
        test_ds = POGFiles(df_test_valid,
                           transform=tfms,n_mels=64,HOP_LEN=1024,N_FFT=512,
                           train=False
                           )

        test_ds.input_path = Path(test_path)

        bs = batch_size
        test_dl = torch.utils.data.DataLoader(test_ds, batch_size=bs*2, num_workers=0, shuffle=False)
        
        preds = []
        probs = []
        for xb  in progress_bar(test_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            probs.append(torch.softmax(output.float(),1).squeeze().cpu())
            preds.append(torch.argmax(output.float(),1).squeeze().cpu())
        
        oof_ = []
        for xb,_  in progress_bar(valid_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            oof_.append(torch.softmax(output.float(),1).squeeze().cpu())
        
        oof_ = [t.detach().numpy() for t in oof_]
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/test_fold_{fold_num}_probs.npy',np.array(probs))
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',oof_)
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',np.array(validation_fold.song_id))        

        preds = torch.cat(preds)
        sample_df = df_test_valid.copy()
        sample_df['target'] = preds
        sample_df.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}.csv', index=False)

In [36]:
df_test_valid.head()

,song_id,filename,filepath
0,7072,007072.ogg,test/007072.ogg
1,10207,010207.ogg,test/010207.ogg
2,20008,020008.ogg,test/020008.ogg
3,10924,010924.ogg,test/010924.ogg
4,21896,021896.ogg,test/021896.ogg


In [37]:
df_test_valid

,song_id,filename,filepath
0,7072,007072.ogg,test/007072.ogg
1,10207,010207.ogg,test/010207.ogg
2,20008,020008.ogg,test/020008.ogg
3,10924,010924.ogg,test/010924.ogg
4,21896,021896.ogg,test/021896.ogg
...,...,...,...
5071,6427,006427.ogg,test/006427.ogg
5072,16903,016903.ogg,test/016903.ogg
5073,1731,001731.ogg,test/001731.ogg
5074,12871,012871.ogg,test/012871.ogg


In [38]:
if __name__=='__main__':
    run()

*****************************************
Training Fold 0
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.963182,2.809966,0.270969,08:09
1,1.798049,1.770393,0.399548,07:51
2,1.732947,1.710223,0.420643,07:49
3,1.719191,1.756083,0.425414,07:50
4,1.628554,1.731318,0.419890,07:49
5,1.553842,1.585597,0.470618,07:51
6,1.411888,1.482889,0.510045,07:50


*****************************************
Training Fold 1
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.969123,2.552113,0.243094,07:31
1,1.777814,1.716916,0.429181,07:51
2,1.710674,2.000475,0.356102,07:52
3,1.617161,1.685778,0.472627,07:48
4,1.569047,1.622118,0.467855,07:48
5,1.483927,1.582433,0.494475,07:48
6,1.326657,1.482821,0.511552,07:51


*****************************************
Training Fold 2
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.964932,2.149576,0.337519,07:33
1,1.798930,1.788093,0.410347,08:21
2,1.694723,1.796166,0.408589,07:49
3,1.601443,1.702381,0.442240,07:49
4,1.544298,1.782367,0.425163,07:51
5,1.460303,1.527187,0.490457,07:53
6,1.283595,1.472278,0.513561,07:52


*****************************************
Training Fold 3
*****************************************
- Training samples: 15927
- Validation Samples : 3981


epoch,train_loss,valid_loss,f1_score,time
0,1.972118,2.307975,0.298669,07:38
1,1.823777,1.742858,0.410198,07:59
2,1.716087,1.872926,0.384326,07:49
3,1.611445,1.614031,0.456167,07:50
4,1.544325,1.645987,0.455413,07:50
5,1.441143,1.572767,0.476262,07:50
6,1.276879,1.491910,0.509169,07:50


*****************************************
Training Fold 4
*****************************************
- Training samples: 15927
- Validation Samples : 3981


epoch,train_loss,valid_loss,f1_score,time
0,1.965258,3.102667,0.203466,07:37
1,1.800006,1.734925,0.415725,17:03
2,1.698467,2.124029,0.401155,12:13
3,1.614815,1.665338,0.448129,07:51
4,1.548935,1.683207,0.447375,07:53
5,1.460996,1.557003,0.492590,07:52
6,1.290065,1.487776,0.513439,07:52


In [39]:
submission

,song_id,genre_id
0,7072,0
1,10207,0
2,20008,0
3,10924,0
4,21896,0
...,...,...
5073,6427,0
5074,16903,0
5075,1731,0
5076,12871,0


### OOF

In [184]:
OOFOut = pd.DataFrame()
for fld in range(5):
    fold_num = fld
    kernel_type= 'tf_efficientnet_b1_MonoSpec'
    OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
    ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
    OOF = ([item for sublist in OOF for item in sublist])
    OOF = pd.DataFrame(OOF)
    OOF['ID'] = ID
    OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)


In [185]:
OOFOut.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,ID
19903,0.578553,0.007145,0.002254,0.022394,0.215048,0.001551,0.041775,0.008167,0.021436,0.026099,0.063606,0.000022,1.773017e-04,0.001083,0.000027,0.007897,0.000034,0.000448,2.283319e-03,2609
19904,0.016311,0.222396,0.099050,0.116055,0.008847,0.275150,0.000634,0.034478,0.102952,0.023702,0.001293,0.003247,5.936155e-04,0.038230,0.020302,0.002589,0.021934,0.012120,1.153403e-04,13023
19905,0.003303,0.583039,0.351965,0.003372,0.000434,0.004570,0.000427,0.008431,0.035841,0.000279,0.000380,0.000048,3.667977e-07,0.001786,0.005143,0.000331,0.000010,0.000641,2.307175e-07,2022
19906,0.429672,0.006871,0.003170,0.025672,0.393051,0.001158,0.072260,0.003579,0.025210,0.017575,0.013816,0.000002,1.330119e-05,0.001230,0.000030,0.005351,0.000031,0.000194,1.114279e-03,9337
19907,0.439870,0.014450,0.002778,0.042812,0.041724,0.003357,0.057617,0.041777,0.025571,0.016857,0.302475,0.001248,5.546088e-04,0.003257,0.000088,0.002868,0.000104,0.000689,1.902413e-03,6579


In [186]:
OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')

In [187]:
OOF_preds=OOFOut.iloc[:,:19].idxmax(1)


In [188]:
from sklearn.metrics import f1_score

f1_score(OOFOut['genre_id'],OOF_preds,average="micro")

0.5115531444645369

In [189]:
OOFOut[OOFOut['genre_id'].isin([17])][17].mean(),OOFOut[~OOFOut['genre_id'].isin([17])][17].mean()

(0.0081252, 0.0025363627)

In [190]:
OOFOut[OOFOut['genre_id'].isin([18])][18].mean(),OOFOut[~OOFOut['genre_id'].isin([18])][18].mean()

(0.0013746043, 0.0004802909)

In [191]:
pd.crosstab(OOF_preds,OOFOut['genre_id'])

genre_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
row_0,,,,,,,,,,,,,,,,,,,
0,2124,192,62,241,393,39,440,137,207,155,479,6,0,26,1,38,1,6,7
1,151,1535,857,198,35,187,20,124,300,59,21,9,0,51,58,14,5,13,0
2,20,817,1467,48,20,16,7,20,61,25,1,0,0,12,13,7,3,7,0
3,212,183,75,871,27,64,63,198,86,42,126,44,12,47,2,2,42,5,2
4,169,19,23,43,1226,10,16,4,24,36,9,1,0,2,1,7,6,4,2
5,27,152,28,76,7,751,4,61,113,52,5,18,1,29,30,5,5,6,1
6,181,16,12,62,13,2,596,9,12,7,15,4,0,1,0,4,0,0,0
7,82,82,12,120,6,48,7,370,37,3,87,56,0,27,2,2,0,3,1
8,14,25,5,7,7,22,3,8,44,8,0,0,0,3,4,1,0,1,0


In [192]:
df = OOFOut.groupby('genre_id').size()
df = 1. / df
df = df / df.mean()

class_weights = df.values
logits_weights = np.log((class_weights).reshape((1, -1)))
probit_weights = scipy.special.softmax(logits_weights,1)

In [193]:
probit_weights

array([[0.00235017, 0.00233194, 0.00279526, 0.00400964, 0.00411011,
        0.00594511, 0.00611123, 0.00691318, 0.00763742, 0.00886654,
        0.00906704, 0.01458053, 0.01768961, 0.02358614, 0.05082648,
        0.07678042, 0.07678042, 0.12443724, 0.55518153]])

In [194]:
# OOFOut.iloc[:,:19] = OOFOut.iloc[:,:19].values+0.05*probit_weights

In [195]:
f1_score(OOFOut['genre_id'],OOFOut.iloc[:,:19].idxmax(1),average="micro")

0.5115531444645369

In [197]:
for a in [13,14,15,16]:
    print(a, OOFOut[OOFOut['genre_id'].isin([a])][a].mean(),OOFOut[~OOFOut['genre_id'].isin([a])][a].mean())

13 0.07008024 0.013286457
14 0.043123726 0.0063900445
15 0.016211083 0.0042011063
16 0.22292155 0.003938406


In [217]:
OOFOut[((OOFOut[[13,14,15,16]]>0.1).sum(1)>0) & (OOFOut['genre_id'].isin([13,14,15,16]))][:15]

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,ID,song_id,genre_id
28,0.021637,0.051874,0.010361,0.080023,0.000271,0.084145,0.004666,0.008223,0.057555,0.437437,...,0.000736,0.168091,0.017800,0.002815,0.001977,0.017576,2.729170e-04,21617,21617,13
123,0.113579,0.075760,0.018706,0.246917,0.004664,0.053977,0.010371,0.061699,0.070461,0.181360,...,0.001042,0.101779,0.006314,0.003467,0.006449,0.012711,2.120956e-03,11952,11952,13
133,0.045694,0.029620,0.014662,0.530355,0.065386,0.042661,0.003145,0.011891,0.045012,0.033541,...,0.000466,0.011537,0.004539,0.000597,0.147284,0.006327,2.526421e-03,2326,2326,16
243,0.004453,0.026902,0.006538,0.040029,0.000151,0.108242,0.000322,0.001712,0.030913,0.603983,...,0.000623,0.111175,0.034916,0.000992,0.004712,0.019504,1.295457e-04,13447,13447,13
434,0.000779,0.000736,0.000571,0.016022,0.488609,0.003914,0.000002,0.000233,0.000940,0.000508,...,0.000230,0.000031,0.000346,0.000003,0.486112,0.000355,5.729354e-04,10930,10930,16
596,0.018877,0.019201,0.002789,0.036063,0.000274,0.080436,0.003894,0.005303,0.053277,0.568265,...,0.000178,0.149929,0.011986,0.004753,0.000922,0.015717,7.244947e-04,517,517,13
619,0.007416,0.061927,0.026014,0.069804,0.014051,0.327785,0.000191,0.004791,0.058576,0.143423,...,0.002339,0.028429,0.099173,0.001281,0.114030,0.038469,8.343680e-04,20936,20936,16
806,0.063104,0.066770,0.030266,0.078113,0.006631,0.040619,0.005463,0.019507,0.075019,0.436695,...,0.001191,0.110311,0.017275,0.014988,0.004176,0.020317,3.812106e-03,66,66,13
861,0.053291,0.158314,0.037514,0.075242,0.000746,0.108086,0.011978,0.051214,0.117971,0.151955,...,0.000343,0.169614,0.017542,0.009948,0.000650,0.016483,3.006123e-04,24416,24416,14
977,0.017421,0.080411,0.030363,0.057116,0.000454,0.070478,0.004208,0.011786,0.044877,0.376967,...,0.008471,0.199211,0.034318,0.005901,0.002527,0.026086,3.627631e-04,24493,24493,13


In [207]:
OOFOut['genre_id']

0        7
1        9
2        0
3        4
4        3
        ..
19903    8
19904    9
19905    1
19906    0
19907    1
Name: genre_id, Length: 19908, dtype: int64

In [218]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/tf_efficientnet_b1_MonoSpec/fold_{fld}.csv')
    out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')

### Mode based sub

In [219]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fld}.csv')
    out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [220]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode.csv',index=False)

### End ###